In [1]:
%load_ext autoreload
%autoreload 2

In [58]:
from scripts.web3_helpers import *
import hmac
import hashlib
import base64
from web3 import Web3
import os, json
from supabase import create_client, Client

In [10]:
w3 = Web3(Web3.HTTPProvider('HTTP://0.0.0.0:8545'))

In [49]:
creds = json.load(open(".db-creds.json"))

url = creds["SUPABASE_URL"]
key = creds["SUPABASE_KEY"]
pw = creds["PW_HASH"]
supabase = create_client(url, key)

In [12]:
def get_data(table_name, select, data):
    initial_len = len(data)
    prev_len = -1
    i = 0
    while (len(data) - initial_len) % 1000 == 0 and prev_len != len(data):
        prev_len = len(data)
        data.extend(supabase.table(table_name).select(select).range(i*1000, i*1000 + 1000).execute().data)
        i += 1
    return data

In [18]:
post_reddit_ids = get_data("posts", 'reddit_id, ipfs_hash', [])
comment_reddit_ids = get_data("comments", "reddit_id, ipfs_hash", [])
categories = get_data("categories", "*", [])
users = get_data("users", "*", [])
follows = get_data("follows", "*", [])
votes = get_data("votes", "*", [])

In [19]:
users[0]

{'id': 2,
 'created_at': '2022-08-05T00:29:01.305294+00:00',
 'username': 'Eabryt',
 'profile_pic': 'https://www.redditstatic.com/avatars/defaults/v2/avatar_default_2.png',
 'msa_id': None,
 'transaction_hash': None,
 'wallet_address_personal': None,
 'wallet_address_provided': None}

In [63]:
user_data = []
wallets = []
for user in users[:10]:
    msa_id = user["msa_id"]
    username = user["username"]
    profile_pic = user["profile_pic"]
    user_wallet = user["wallet_address_provided"]
    if user_wallet is None:
        dig = hmac.new(pw.encode(), msg="PocketAAcez".encode(), digestmod=hashlib.sha256).digest()
        pw_hash = base64.b64encode(dig).decode()
        user_wallet = w3.eth.account.from_key(keccak(encode_abi_packed(['string'],[f"{pw_hash}"])))
        
    wallets.append(user_wallet)
    user_data.append('{' + f'"msa_id": {msa_id},"username": "{username}","profile_pic": "{profile_pic}","wallet_address": "{user_wallet.address}"' + '}')

In [64]:
create_msas_with_delegator(wallets)

EncodingTypeError: Value `None` of type <class 'NoneType'> cannot be encoded by PackedUnsignedIntegerEncoder

In [45]:
dig = hmac.new(b'password', msg="PocketAAcez".encode(), digestmod=hashlib.sha256).digest()
base64.b64encode(dig).decode()

'7DR6F/yY56zt9XVYrB4ogIYAo76hlPgL5udpXxgAgr8='

In [75]:
delegate_msa_id = get_msa_id(delegate)

that execution reverted: VM Exception while processing transaction: revert User does not have msaId
that execution reverted: VM Exception while processing transaction: revert User does not have msaId


In [76]:
delegate_msa_id